In [31]:
import pickle 
import numpy as np
import pandas as pd
import h5py
from sklearn.model_selection import train_test_split

In [3]:
with open('all_shots_with_T_raw_p3.pickle', 'rb') as file:
    pulse_dicts = pickle.load(file)
    
"""
with open('all_shots_with_T_raw_p3.pickle', 'wb') as file:
    pickle.dump(pulse_dicts, file, protocol=pickle.HIGHEST_PROTOCOL)
"""

In [4]:
input_cols_efit = ['Q95', 'RGEO', 'CR0', 'VOLM', 'TRIU', 'TRIL', 'XIP', 'ELON', 'POHM']
input_cols_scal = ['BT']
input_cols_gash = ['ELER']
input_cols_nbi = ['P_NBI']
input_cols_icrh = ['P_ICRH']

col_time_efit = ['EFIT_T']

In [5]:
f = h5py.File('../processed/profile_database.hdf5', 'w')
for n, (pulse_id, pulse_data) in enumerate(pulse_dicts.items()):
    try :
        pulse_grp = f.create_group(pulse_id)

        input_grp = pulse_grp.create_group('machine_parameters')
        profile_grp = pulse_grp.create_group('profiles')

        pulse_input_data = pulse_data['inputs']
        pulse_output_data = pulse_data['outputs']

        for input_id, input_data in pulse_input_data.items():
            if input_id == 'EFIT_T' or input_id ==  'CRO':
                continue
            individual_grp = input_grp.create_group(input_id)
            
            if input_id not in input_cols_efit:
                input_data, input_time = input_data['values'], input_data['time']
            else:
                input_time = pulse_input_data['EFIT_T']
            
            dset_r = individual_grp.create_dataset("values", data=np.array(input_data))
            dset_t = individual_grp.create_dataset("time", data=np.array(input_time))
            
        for out_id, output_data in pulse_output_data.items():
            dset_p = profile_grp.create_dataset(out_id, data=output_data)
            
        
    except ValueError as e: 
        print(e)
f.close() 

In [38]:
def printname(name):
    return name

i = 0
with h5py.File('../processed/profile_database.hdf5', 'r') as file: 
    # print(file.visit(printname))
    
    for name in file: 
        print(name)
        for subgroup in file[name]:
            print(subgroup)
            for index in file[name + '/' + subgroup]:
                print(index)
                for field in file[name + '/' + subgroup + '/' + index]:
                    print(field)
        break 
        # i += 1
        # if i == 3: 
        #     break
    # for pulse in file.keys():
    #     print(file[pulse + '/machine_parameters'].keys())
    #    break

79015
machine_parameters
BT
time
values
CR0
time
values
ELER
time
values
ELON
time
values
POHM
time
values
P_ICRH
time
values
P_NBI
time
values
Q95
time
values
RGEO
time
values
TRIL
time
values
TRIU
time
values
VOLM
time
values
XIP
time
values
profiles
DNE
[1.0552838e+20 1.0552838e+20 1.0552838e+20 1.0552838e+20 1.0552838e+20
 1.0552838e+20 1.0552838e+20 1.0552838e+20 1.0552838e+20 1.0552838e+20
 1.0552838e+20 1.0552838e+20 1.0552838e+20 1.0552838e+20 1.8846870e+20
 1.0552838e+20 1.0552838e+20 1.0552838e+20 1.0552838e+20 1.0552838e+20
 1.0552838e+20 1.0552838e+20 1.0552838e+20 1.0552838e+20 1.0552838e+20
 1.0552838e+20 1.0552838e+20 1.0552838e+20 1.7363957e+19 1.0552838e+20
 1.0552838e+20 1.0552838e+20 1.0552838e+20 1.0552838e+20 1.0552838e+20
 1.0552838e+20 1.0552838e+20 7.1705786e+19 1.0552838e+20 1.0552838e+20
 1.0552838e+20 1.0552838e+20 1.0552838e+20 1.0552838e+20 1.0552838e+20
 1.0552838e+20 1.0552838e+20 1.0552838e+20 2.4950299e+18 1.0552838e+20
 1.0552838e+20 1.0552838e+20 1.05

 1.0552838e+20 1.0552838e+20 1.0552838e+20]
[2.4020231e+18 2.1553314e+18 1.8178029e+18 1.8609034e+18 1.8744590e+18
 1.6897565e+18 2.0962658e+18 2.2761452e+18 1.6569005e+18 1.9474790e+18
 1.7785784e+18 1.8044797e+18 2.3175495e+18 2.2143584e+18 2.0183694e+18
 1.8189134e+18 1.8176120e+18 1.5716664e+18 1.8117983e+18 1.8381319e+18
 1.8936226e+18 2.1500164e+18 2.3172254e+18 2.0203240e+18 2.1161401e+18
 2.1932342e+18 1.7850334e+18 2.1885155e+18 2.2513467e+18 1.8376102e+18
 2.2304881e+18 2.2000637e+18 1.7956711e+18 2.1629781e+18 2.2274179e+18
 1.5393449e+18 2.0992115e+18 2.8447513e+18 1.8229616e+18 2.2539105e+18
 2.4281354e+18 2.1068889e+18 2.3833528e+18 2.1778200e+18 1.7339058e+18
 2.6822394e+18 2.6784051e+18 2.2647705e+18 3.4798936e+18 4.0284003e+18
 3.7482882e+18 3.2227137e+18 2.6845888e+18 2.9887887e+18 3.2293003e+18
 4.1297459e+18 3.1492677e+18 2.9034760e+18 1.0552838e+20 8.0955172e+17
 1.0552838e+20 1.0552838e+20 1.0552838e+20]
[2.3124962e+18 2.1422193e+18 1.7498857e+18 2.1060356e+18 2.1

 1.0552838e+20 1.0552838e+20 1.0552838e+20]
[3.5184993e+18 3.1268800e+18 2.5292847e+18 2.7559190e+18 2.7754890e+18
 2.5718877e+18 2.9999380e+18 2.9110178e+18 2.7439354e+18 2.9270492e+18
 2.7146802e+18 2.7038112e+18 3.3237071e+18 3.1555327e+18 2.8716259e+18
 2.8236696e+18 2.7757133e+18 2.7277570e+18 2.6798007e+18 2.7769420e+18
 2.9996648e+18 3.4136120e+18 3.5205224e+18 2.9564639e+18 3.7170599e+18
 3.9691204e+18 3.0846150e+18 3.3672351e+18 3.4235683e+18 2.6998687e+18
 3.5785533e+18 3.8329399e+18 3.1417377e+18 4.6996948e+18 3.9906942e+18
 2.6673905e+18 2.9858593e+18 1.0552838e+20 2.9633763e+18 3.8386246e+18
 4.1662150e+18 3.6936667e+18 3.6224843e+18 3.4850090e+18 2.9017976e+18
 3.7351988e+18 4.8425686e+18 4.5631159e+18 4.3596290e+18 4.8799311e+18
 4.9991721e+18 4.3265038e+18 4.4810333e+18 4.7074540e+18 4.1822860e+18
 1.0552838e+20 4.8127361e+18 3.4714251e+18 8.7614271e+18 5.8104940e+18
 3.2392643e+18 2.0825409e+18 1.0552838e+20]
[4.0351117e+18 3.7342730e+18 2.9405809e+18 3.2246821e+18 3.3

[ 376.508     377.00464   248.54839   301.7752    349.96503   306.9446
  384.70615   399.56345   257.5809    272.28995   240.72438   255.78271
  284.87216   272.16287   248.68494   339.21298   306.39368   235.0896
  238.2963    213.41592   233.1491    216.64938   222.98032   190.76994
  257.80255   244.3999    218.98853   237.80185   236.80911   198.83487
  218.51906   282.76904   162.53804   230.44427   149.09937   137.50769
  189.06877   197.01447   155.6078    211.94832   196.5664    139.28174
  184.25618   183.0833    121.23592   235.97276   188.5161    127.201256
  108.08901   129.29599   124.359     142.63414   152.09566   115.004745
  117.15832   111.93037    68.58098    64.4409     58.623833 1000.
 1000.       1000.       1000.      ]
[ 368.52463   340.79825   345.53714   356.51376   368.14056   341.12457
  344.73233   386.05692   298.01642   307.1325    318.56604   315.78705
  342.77148   289.70908   251.57503   309.16147   335.3671    262.46832
  196.80876   232.77298   210.4

 2.3843234e+18 0.0000000e+00 0.0000000e+00]
[1.5826432e+19 1.6802407e+19 1.6912126e+19 1.5667148e+19 1.4513357e+19
 1.5965142e+19 1.5081847e+19 1.5703874e+19 1.4514919e+19 1.4074959e+19
 1.4119574e+19 1.3480835e+19 1.5450452e+19 1.3646469e+19 1.3917106e+19
 1.3079015e+19 1.3068270e+19 1.4103105e+19 1.4512394e+19 1.3974689e+19
 1.2808873e+19 1.3138480e+19 1.0716841e+19 1.1995522e+19 1.1505353e+19
 1.1427091e+19 1.2392469e+19 1.1533034e+19 1.1403463e+19 1.2169264e+19
 1.1171535e+19 1.0695273e+19 9.7000730e+18 1.2980388e+19 7.2733843e+18
 9.4551293e+18 1.2006992e+19 1.0285058e+19 8.6378744e+18 9.5114463e+18
 8.3120941e+18 8.3551306e+18 7.5353226e+18 6.4128988e+18 7.4088436e+18
 9.0040865e+18 8.0879531e+18 6.6558524e+18 5.2327891e+18 6.7106345e+18
 6.9554677e+18 8.8652325e+18 4.0556822e+18 5.3795932e+18 3.9741364e+18
 3.0342156e+18 5.6612210e+18 7.4251065e+18 4.9682159e+18 0.0000000e+00
 5.5983058e+18 3.6472587e+18 1.3386455e+18]
[1.5450798e+19 1.5859878e+19 1.6388800e+19 1.5480380e+19 1.6

 5.8239014e+18 0.0000000e+00 0.0000000e+00]
[4.3641371e+19 4.0075303e+19 4.2616349e+19 3.9066369e+19 4.0865351e+19
 4.1014304e+19 4.0627940e+19 3.9804656e+19 4.1631328e+19 3.8748074e+19
 4.1772501e+19 3.8850544e+19 3.8334530e+19 3.8949914e+19 4.1088116e+19
 3.9929996e+19 4.1239915e+19 4.0724327e+19 3.8874852e+19 3.9560345e+19
 3.7510723e+19 3.8282066e+19 3.9429208e+19 3.7340457e+19 3.9277674e+19
 3.9812283e+19 3.7841522e+19 3.7600096e+19 3.5920423e+19 3.6249210e+19
 3.7520962e+19 3.3567474e+19 3.5148253e+19 3.3749017e+19 3.5846322e+19
 3.4789256e+19 3.4764143e+19 3.5829999e+19 3.3800810e+19 3.6544233e+19
 3.3524607e+19 3.1047869e+19 3.1132755e+19 3.2231792e+19 3.0104356e+19
 3.0068417e+19 3.0050537e+19 3.2968456e+19 2.6793453e+19 2.6367465e+19
 3.2236410e+19 2.6605835e+19 2.5161000e+19 2.5721542e+19 2.7062748e+19
 2.0081562e+19 2.1601909e+19 1.2164027e+19 0.0000000e+00 0.0000000e+00
 0.0000000e+00 3.5779708e+18 0.0000000e+00]
[4.2218814e+19 4.1708650e+19 3.7816884e+19 4.0084759e+19 4.2

  597.8573    413.5377      0.      ]
[1785.7577   1909.0267   2219.1724   2075.4392   1611.213    1902.8976
 2031.5737   2335.689    1865.4198   1691.3024   1682.3933   1723.6184
 1958.4608   1653.9266   2014.6517   1747.0594   1603.3862   1630.8157
 1692.9803   1570.088    1450.8606   1382.962    1744.3052   1471.3749
 1220.372    1060.2432   1197.4691    834.92206  1064.7865   1073.652
  955.00995   960.6282    947.79895   545.9259    638.1873    617.83655
  857.6992    675.17566   703.4353    435.4273    479.32446   375.35684
  398.327     473.24066   391.59125   312.0208    353.4681    287.64804
  205.70987   159.59126   173.98792   285.2838    201.38536   240.36815
  180.72887   231.19827    82.315445  274.8087    163.06264    75.16067
    0.          0.         24.98691 ]
[2089.5264   2246.9148   2010.7994   1957.498    1976.7712   1988.0907
 1756.8469   1922.9812   1793.7599   1847.9484   1799.6505   2017.3862
 1904.3145   1786.4496   1780.3475   1746.4209   1625.4064   1850.60

[1602.8434   1251.1869   1335.5804   1282.4231   1423.7661   1412.2119
 1218.5762   1169.9037   1228.5312   1236.3822   1323.8865   1246.0391
 1307.0867   1263.807    1145.2855   1139.0758   1132.866    1126.6562
 1120.4465   1020.3462    943.1335    964.8338   1036.9662    859.84973
  819.25867   760.9727    736.47565   733.90314   685.09985   738.76636
  823.0125   1131.0472    635.8752    482.01932   648.5234    549.5579
  548.3969    538.3105    537.0007    455.96338   454.33752   308.0306
  317.47693   312.61243   243.19101   313.0825    196.2564    208.40881
  211.88484   119.27978   129.90305   125.89249    92.74176    83.06631
   89.37306    75.35981    57.232548   25.064318   58.944828   19.707748
    0.          0.          0.      ]
[1257.0171   1444.5745   1277.8143   1297.1013   1488.5674   1316.3331
 1221.3412   1586.266    1234.9935   1249.3198   1362.5981   1214.3668
 1398.4176   1302.0681   1329.2368   1272.7582   1216.2795   1159.8009
 1103.3223   1088.4211    916.154

67.07973
67.12975
67.17978
67.229805
67.27983
67.32986
67.37988
67.42991
67.479935
67.52996
67.57999
67.63001
67.68004
67.730064
67.78008
67.83011
67.880135
67.93016
67.98019
68.03021
68.08024
68.130264
68.18029
68.230316
68.28034
68.33037
68.380394
68.43042
68.480446
68.53047
68.5805
68.63052
68.68055
68.730576
68.7806
68.83063
68.88065
68.93068
68.980705
69.03073
69.08076
69.130775
69.18081
69.230835
69.28086
69.33089
69.38091
69.43093
69.48096
69.53098
69.58101
69.631035
69.68106
69.73109
69.78111
69.83114
69.881165
69.93119
69.98122
70.03124
70.08127
70.131294
70.18132
70.231346
70.28137
70.3314
70.381424
70.43145
70.481476
70.5315
70.58153
70.63155
70.68158
70.731606
70.78163
70.83166
70.88168
70.93171
70.981735
71.03176
71.08179
71.13181
71.18184
71.231865
71.28189
71.33192
71.38194
71.43197
71.481995
71.53202
71.58205
71.63207
71.6821
71.732124
71.78215
71.83218
71.8822
71.93222
71.98225
72.03227
72.0823
72.132324
72.18235
72.232376
72.2824
72.33243
72.382454
72.43248
72.482506


Now we can create the PSI database

In [40]:
EFIT = ['Q95', 'RGEO', 'CR0', 'VOLM', 'TRIU', 'TRIL', 'XIP', 'ELON', 'POHM']
ALL_KEYS = EFIT + ['BT', 'ELER', 'P_NBI', 'P_ICRH', 'NE']
def sample_nesep(mean_val, std_val):
    return np.random.normal(mean_val, std_val)

def sample_input(input_grp, key, t1, t2, nesep):
    # for key in list(input_grp.keys()):
    if key == 'NE':
        return nesep
    source = key + '/'
    sample_time = input_grp[key + '/time'][:]
    sample_values = input_grp[key + '/values'][:]

    time_idx = np.logical_and(sample_time > t1, sample_time < t2)
    sample = np.mean(sample_values[time_idx])
        
    return sample

def get_ne_and_te_profiles(raw_shot, t1, t2):
    sample_ne = raw_shot['profiles/NE'][:]
    sample_te = raw_shot['profiles/TE'][:]
    sample_time = raw_shot['profiles/time'][:]
    sample_radius = raw_shot['profiles/radius'][:]
    
    profiles_idx = np.logical_and(sample_time > t1, sample_time < t2)
    profiles_ne = sample_ne[profiles_idx]
    profiles_te = sample_te[profiles_idx]
    if len(profiles_ne[0]) != 63:
        profiles_ne = np.pad(profiles_ne, ((0, 0), (0, 63 - len(profiles_ne[0]))), 'constant')
    if len(profiles_te[0]) != 63:
        profiles_te = np.pad(profiles_te, ((0, 0), (0, 63 - len(profiles_te[0]))), 'constant')
    
    # Returns to numpy arrays of the same shape
    return profiles_ne, profiles_te, sample_radius

def combine_ne_te(prof_ne, prof_te):
    
    combined = np.stack([prof_ne, prof_te], axis=1)
    # Want a shape of num_slices X 63
    return combined

def sample_slices(slices, y):
    num_windows = len(slices)
        
    test_size = int(0.2*num_windows)
    val_size = int(0.1*num_windows)
    
    if val_size == 0 or test_size==0:
        val_size = 1
        test_size = 1
        
    train_size = num_windows - test_size - val_size
    
    X_train, X_test, y_train, y_test, train_idx, test_idx =  train_test_split(slices, y, range(len(slices)), test_size=test_size)
    X_train, X_val, y_train, y_val, train_idx, val_idx = train_test_split(X_train, y_train, range(len(X_train)), test_size=val_size)
    
    return X_train, X_test, X_val, y_train, y_test, y_val, train_idx, val_idx, test_idx

In [27]:
jet_pdb = pd.read_csv('pedestal-database.csv')
important_info = jet_pdb[['shot', 't1', 't2', 'neseparatrixfromexpdata10^19(m^-3)', 'error_neseparatrixfromexpdata10^19(m^-3)','neseparatrixfromfit10^19(m^-3)',
 'error_neseparatrixfromfit10^19(m^-3)','FLAG:HRTSdatavalidated']]
final_pulse_list = important_info[(important_info['neseparatrixfromexpdata10^19(m^-3)'] != 0.0) & (important_info['neseparatrixfromexpdata10^19(m^-3)'] != -1.0) &  (important_info['shot'] >= 79000) &  (important_info['FLAG:HRTSdatavalidated'] > 0)]


In [ ]:
X_train_str, y_train_str = [], []
X_val_str, y_val_str = [], []
X_test_str, y_test_str = [], []

X_train_both, y_train_both = [], []
X_val_both, y_val_both = [], []
X_test_both, y_test_both = [], []

radius_train_str = []
radius_val_str = []
radius_test_str = []
radius_train_both = []
radius_val_both = []
radius_test_both = []

pulse_list = []
with h5py.File('../processed/profile_database.hdf5', 'r+') as f:
    try: 
        grp_datasets = f.create_group("processed_datasets")
    except ValueError as e: 
        grp_datasets = f['processed_datasets']
    try: 
        grp_psi = grp_datasets.create_group("PSI22")
    except ValueError as e: 
        grp_psi = grp_datasets['PSI22']
    
    try: 
        grp_strohman = grp_psi.create_group("density")
    except ValueError as e: 
        grp_strohman = grp_psi['density']
    try: 
        grp_both = grp_psi.create_group("density_and_temperature")
    except ValueError as e: 
        grp_both = grp_psi['density_and_temperature']
    
    try: 
        grp_str_train = grp_strohman.create_group('train')
        grp_str_test = grp_strohman.create_group('test')
        grp_str_val = grp_strohman.create_group('valid')

        grp_both_train = grp_both.create_group('train')
        grp_both_test = grp_both.create_group('test')
        grp_both_val = grp_both.create_group('valid')
    except: 
        grp_str_train = grp_strohman['train']
        grp_str_test = grp_strohman['test']
        grp_str_val = grp_strohman['valid']
        grp_both_train = grp_both['train']
        grp_both_test = grp_both['test']
        grp_both_val = grp_both['valid']

    # loop through JET PDB 
    for index, row in final_pulse_list.iterrows():
        shot, t1, t2 = str(int(row['shot'])), row['t1'], row['t2']
        nesep, dnesep = row['neseparatrixfromexpdata10^19(m^-3)'], row['error_neseparatrixfromexpdata10^19(m^-3)']
        raw_shot = f[shot]
        pulse_list.append(shot)
        
        profiles_ne, profiles_te, radii = get_ne_and_te_profiles(raw_shot, t1, t2)
        combined = combine_ne_te(profiles_ne, profiles_te)
        other_params = np.array([[sample_input(raw_shot['machine_parameters'], key, t1, t2, nesep) for key in ALL_KEYS] for _ in range(len(profiles_ne))])
        
        
        ne_train, ne_test, ne_valid, ne_y_train, ne_y_test, ne_y_val, train_idx, val_idx, test_idx = sample_slices(profiles_ne, other_params)
        
        both_train, both_test, both_valid, both_y_train, both_y_test, both_y_val, both_train_idx, both_val_idx, both_test_idx = sample_slices(combined, other_params)
        
        X_train_str.extend(ne_train)
        X_val_str.extend(ne_valid)
        X_test_str.extend(ne_test)
        y_train_str.extend(ne_y_train)
        y_val_str.extend(ne_y_val)
        y_test_str.extend(ne_y_test)


        X_train_both.extend(both_train)
        X_val_both.extend(both_valid)
        X_test_both.extend(both_test)
        y_train_both.extend(both_y_train)
        y_val_both.extend(both_y_val)
        y_test_both.extend(both_y_test)
        
        radius_train_str.extend([radii for _ in range(len(ne_train))])
        radius_val_str.extend([radii for _ in range(len(ne_valid))])
        radius_test_str.extend([radii for _ in range(len(ne_test))])
        radius_train_both.extend([radii for _ in range(len(both_train))])
        radius_val_both.extend([radii for _ in range(len(both_valid))])
        radius_test_both.extend([radii for _ in range(len(both_test))])
    
    try: 
        meta_group = grp_psi.create_group("meta")
    except ValueError as e: 
        meta_group = grp_psi['meta']
    pulse_list = np.array(list(set([int(key) for key in pulse_list])))
    key_list = np.array([s.encode('utf-8') for s in ALL_KEYS])
    meta_pulse = meta_group.create_dataset('pulse_list', data=pulse_list)
    meta_y_atrr = meta_group.create_dataset('y_column_names', data=key_list)

    print(np.vstack(X_train_str).shape, np.vstack(y_train_str).shape)
    print(np.vstack(X_val_str).shape, np.vstack(y_val_str).shape)
    print(np.vstack(X_test_str).shape, np.vstack(y_test_str).shape)

    print(np.stack(X_train_both).shape, np.stack(y_train_both).shape)
    print(np.stack(X_val_both).shape, np.stack(y_val_both).shape)
    print(np.stack(X_test_both).shape, np.stack(y_test_both).shape)
    dset_X_train_str = grp_str_train.create_dataset("X", data=X_train_str)
    dset_y_train_str = grp_str_train.create_dataset("y", data=y_train_str)
    dset_r_train_str = grp_str_train.create_dataset("radii", data=radius_train_str)

    dset_X_test_str = grp_str_test.create_dataset("X", data=X_test_str)
    dset_y_test_str = grp_str_test.create_dataset("y", data=y_test_str)
    dset_r_test_str = grp_str_test.create_dataset("radii", data=radius_test_str)

    dset_X_val_str = grp_str_val.create_dataset("X", data=X_val_str)
    dset_y_val_str = grp_str_val.create_dataset("y", data=y_val_str)
    dset_r_val_str = grp_str_val.create_dataset("radii", data=radius_val_str)


    dset_X_train_both = grp_both_train.create_dataset("X", data=X_train_both)
    dset_y_train_both = grp_both_train.create_dataset("y", data=y_train_both)
    dset_r_train_both = grp_both_train.create_dataset("radii", data=radius_train_both)
        
    dset_X_test_both = grp_both_test.create_dataset("X", data=X_test_both)
    dset_y_test_both = grp_both_test.create_dataset("y", data=y_test_both)
    dset_r_test_both = grp_both_test.create_dataset("radii", data=radius_test_both)
    
    dset_X_val_both = grp_both_val.create_dataset("X", data=X_val_both)
    dset_y_val_both = grp_both_val.create_dataset("y", data=y_val_both)
    dset_r_val_both = grp_both_val.create_dataset("radii", data=radius_val_both)